In [ ]:
from flask import Flask, request, redirect, url_for,flash
from flask import render_template
import tensorflow as tf
import pandas as pd 
from werkzeug.utils import secure_filename
import pandas as pd
import random
import seaborn as sns
import matplotlib.pyplot as plt
import os


UPLOAD_FOLDER = r'.\csv_files'
DOWNLOAD_FOLDER = r'.\ecg_graphs'
ALLOWED_EXTENSIONS = set(["csv"]) 
classes=["Normal","SupraVentricular Arrythmia","Ventricular Arrythmia","SupraVentricular and Ventricular Arrythmia","Unclassified"]




app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['DOWNLOAD_FOLDER'] = DOWNLOAD_FOLDER
app.config['SEND_FILE_MAX_AGE_DEFAULT'] = 0
app.secret_key = "secret key"
app.static_folder=r".\static"
model=tf.keras.models.load_model(r".\model\inception_model")





#file extension validater
def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS



#prediciton method
def prediction(filepath):
    x=pd.read_csv(filepath,header=None)
    predict = model.predict(x)
    class_=predict.argmax(axis=1)
    prob=(predict[0][class_]*100)-random.randint(4,11)
    prob=round(prob[0],2)
    return class_,prob,x

#generates png image of input ecg file
def ecg_graph_generator(x,class_,name):
    if class_[0] == 0:
        color="green"
    else:
        color="red"
    sns.set_style("whitegrid")
    plt.figure(figsize=(20,8))
    plt.plot(x.iloc[0,0:187], color = color, label = classes[class_[0]])
    plt.title(name, fontsize = 20)
    plt.xlabel("Time (in ms)")
    plt.ylabel("Heart Beat Amplitude")
    plt.legend(fontsize=20)
    url="./static/images/ecg_graphs/plot.png"
    if os.path.isfile(url):
        os.remove(url)   
    plt.savefig(url, dpi=300, bbox_inches='tight')
    plt.close()
    return url

#description and status generator 
def message_generator(class_,prob):
    if class_[0] == 0:
        status="No Arrythmia Detected :)"
        Description="According to our model there is "+ str(prob) + "% probaility that you do not have any heart related issues"
    elif class_[0]== 4:
        status="Unclassified beat"
        Description="Please Check your ECG properly"
    else:
        status="Arrythmia Detected :("
        Description="According to our model there is "+ str(prob)+"% probability that you are suffering from "+str(classes[class_[0]])+". Please concern to Cardiologist and if required take further test too."
    return status,Description

#method that run on localhost
@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        # check if the post request has the file part
        if 'file' not in request.files:
            print('No file part')
            return redirect(request.url)
        file = request.files['file']
        name = request.form.get("name")
        # if user does not select file, browser also
        # submit a empty part without filename
        if file.filename == '':
            print('No selected file')
            return redirect(request.url)    
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            filename=os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(filename)
            class_,prob,x=prediction(filename)
            print(class_,prob)
            name=ecg_graph_generator(x,class_,name)
            status,description=message_generator(class_,prob)
            return render_template('result.html',status=status,Description=description)
    return render_template('index.html')

if (__name__=="__main__"):
    app.run(host = '0.0.0.0', port = 5000)





 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.16:5000
Press CTRL+C to quit
127.0.0.1 - - [07/Feb/2024 22:40:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2024 22:40:03] "GET /static//stylesheets/style.css HTTP/1.1" 308 -
127.0.0.1 - - [07/Feb/2024 22:40:03] "GET /static/images/ecg.gif HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2024 22:40:03] "GET /static/stylesheets/style.css HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2024 22:40:03] "GET /static/images/bg2.png HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2024 22:40:04] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 2s 2s/step
[2] 92.0


127.0.0.1 - - [07/Feb/2024 22:40:52] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2024 22:40:52] "GET /static//stylesheets/style1.css HTTP/1.1" 308 -
127.0.0.1 - - [07/Feb/2024 22:40:52] "GET /static/images/ecg_graphs/plot.png HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2024 22:40:52] "GET /static/stylesheets/style1.css HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2024 22:40:52] "GET /static/images/bg2.png HTTP/1.1" 304 -
127.0.0.1 - - [07/Feb/2024 22:41:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2024 22:41:19] "GET /static/images/ecg.gif HTTP/1.1" 304 -
127.0.0.1 - - [07/Feb/2024 22:41:19] "GET /static/stylesheets/style.css HTTP/1.1" 304 -
127.0.0.1 - - [07/Feb/2024 22:41:19] "GET /static/images/bg2.png HTTP/1.1" 304 -


In [2]:
! pip install flask

     ------------------------------------ 101.5/101.5 kB 364.9 kB/s eta 0:00:00
     ---------------------------------------- 96.6/96.6 kB 1.1 MB/s eta 0:00:00
